# Classificação de clientes em uma empresa automobilística

## Enunciado

Selecionar 1 conjunto de dados que contenha ao menos 4 classes. Aplicar sobre o conjunto os seguintes algoritmos: árvore, bagging, boosting (pode ser o AdaBoost), Random Forest. Avaliar quais deles gera o melhor modelo usando para tanto alguma técnica de validação (holdout, etc.). Escolha uma medida para avaliar o desempenho (acurácia, F1, etc.). Considerando o de melhor resultado, faça a setagem de hiperparâmetros no mesmo para ver se o desempenho melhora.

Transformar o problema multiclasse em problemas binários usando OVA e OVO. Escolha uma medida para avaliar o desempenho (acurácia, F1, etc.) e diga qual das duas abordagens se saiu melhor usando para tanto alguma técnica de validação (holdout, etc.). Use como algoritmo o de árvore.

## Sobre os dados

Os dados foram extraídos de um dataset compartilhado no Kaggle pelo usuário Kash e obtido pelo Analytics Vidhya hackathon, disponível em https://www.kaggle.com/datasets/kaushiksuresh147/customer-segmentation.

O estudo é baseando em um experimento anterior, em que clientes de um mercado existente foram classificados em 4 segmentos distintos de acordo com algumas de suas características. Assim, um atendimento mais direcionado foi proposto para cada segmento, gerando bons resultados.

A ideia é repetir o mesmo experimento para novos mercados e novos clientes.

As variáveis obtidas são:

* ID: número de ID.

* Gender: gênero do cliente (feminino ou masculino).

* Ever_Married: estado civil do cliente (se já se casou ou não).

* Age: idade do cliente.

* Graduated: se o cliente é graduado ou não.

* Profession: profissão do cliente.

* Work_Experience: experiência profissional em anos.

* Spending_Score: pontuação de gastos no mercado.

* Family_Size: tamanho da família.

* Var_1: variável categórica mantida em anonimato.

* Segmentation: Segmento do cliente (variável-alvo).

## Pré-processamento dos dados

Pacotes utilizados:

In [77]:
# Para manipulação de dados
import pandas as pd 

# Para fazer encoding de variáveis categóricas
from sklearn.preprocessing import LabelEncoder

# Modelos de classificação utilizados
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

# Para realizar a avaliação via cross validation
from sklearn.model_selection import cross_val_score

# Estratégias para decomposição em um problema de classe binário
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier

# Para fazer uma busca entre hiperparâmetros
from sklearn.model_selection import GridSearchCV




Já são oferecidas duas planilhas de dados, sendo uma para treino e outra para teste. Mas estarei juntando as duas para fazer o pré-processamento em conjunto.

In [78]:
# Juntando os dados para fazer limpezas

df_treino = pd.read_csv("Train.csv", sep=",")
df_teste = pd.read_csv("Test.csv", sep=",")
df = pd.concat([df_treino, df_teste])

df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [79]:
# Checando os tipos de variáveis

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10695 entries, 0 to 2626
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               10695 non-null  int64  
 1   Gender           10695 non-null  object 
 2   Ever_Married     10505 non-null  object 
 3   Age              10695 non-null  int64  
 4   Graduated        10593 non-null  object 
 5   Profession       10533 non-null  object 
 6   Work_Experience  9597 non-null   float64
 7   Spending_Score   10695 non-null  object 
 8   Family_Size      10247 non-null  float64
 9   Var_1            10587 non-null  object 
 10  Segmentation     10695 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1002.7+ KB


### Limpeza de valores ausentes

In [80]:
# Checando se há valores ausentes

df.isna().sum()

ID                    0
Gender                0
Ever_Married        190
Age                   0
Graduated           102
Profession          162
Work_Experience    1098
Spending_Score        0
Family_Size         448
Var_1               108
Segmentation          0
dtype: int64

In [81]:
# Faremos uma remoção desses valores

df.dropna(inplace=True)
df.isna().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64

### Limpeza de valores duplicados

In [82]:
# Verificamos quantos valores duplicados existem

df.duplicated().sum()


26

In [83]:
# Removendo os valores duplicados

df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

### Encoding de variáveis categóricas

Vamos verificar os valores que cada variável categórica pode assumir para fazer um encoding adequado para cada uma.

In [84]:
print(df['Gender'].unique())
print(df['Ever_Married'].unique())
print(df['Graduated'].unique())
print(df['Profession'].unique())
print(df['Spending_Score'].unique())
print(df['Var_1'].unique())

['Male' 'Female']
['No' 'Yes']
['No' 'Yes']
['Healthcare' 'Engineer' 'Lawyer' 'Artist' 'Doctor' 'Homemaker'
 'Entertainment' 'Marketing' 'Executive']
['Low' 'High' 'Average']
['Cat_4' 'Cat_6' 'Cat_7' 'Cat_3' 'Cat_1' 'Cat_2' 'Cat_5']


Podemos observar que as variáveis Gender, Ever_Married e Graduated podem ser transformados em variáveis binárias. Além disso, a variável Var_1 pode receber valores em sua ordem alfabética (ou seja, 'Cat_1' será transformado em 0, 'Cat_2' será transformado em 1...).

Podemos fazer essas transformações seguindo o comando abaixo:

In [85]:
# Encoding nas variáveis categóricas em ordem alfabética

label_encoders = {}
categorical_columns = ['Gender', 'Ever_Married', 'Graduated', 'Var_1']
for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,1,0,22,0,Healthcare,1.0,Low,4.0,3,D
2,466315,0,1,67,1,Engineer,1.0,Low,1.0,5,B
3,461735,1,1,67,1,Lawyer,0.0,High,2.0,5,B
5,461319,1,1,56,0,Artist,0.0,Average,2.0,5,C
6,460156,1,0,32,1,Healthcare,1.0,Low,3.0,5,C


Para a variável Spending_Score fazemos um encoding seguindo a ordem de suas variáveis.

In [86]:
mapeamento = {'Low': 0, 'Average': 1, 'High': 2}

# Aplicar o mapeamento usando a função map()
df['Spending_Score'] = df['Spending_Score'].map(mapeamento)

df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,1,0,22,0,Healthcare,1.0,0,4.0,3,D
2,466315,0,1,67,1,Engineer,1.0,0,1.0,5,B
3,461735,1,1,67,1,Lawyer,0.0,2,2.0,5,B
5,461319,1,1,56,0,Artist,0.0,1,2.0,5,C
6,460156,1,0,32,1,Healthcare,1.0,0,3.0,5,C


Por fim, como a variável categórica Profession não é ordinal e possui diversos valores, acredito que a abordagem mais adequada seja opção seja aplicar o One-Hot encoding nesta variável.

In [87]:
df = pd.get_dummies(df, columns=['Profession'])
df.head()


,ID,Gender,Ever_Married,Age,Graduated,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,Profession_Lawyer,Profession_Marketing
0,462809,1,0,22,0,1.0,0,4.0,3,D,0,0,0,0,0,1,0,0,0
2,466315,0,1,67,1,1.0,0,1.0,5,B,0,0,1,0,0,0,0,0,0
3,461735,1,1,67,1,0.0,2,2.0,5,B,0,0,0,0,0,0,0,1,0
5,461319,1,1,56,0,0.0,1,2.0,5,C,1,0,0,0,0,0,0,0,0
6,460156,1,0,32,1,1.0,0,3.0,5,C,0,0,0,0,0,1,0,0,0


Por fim, fazemos a separação das variáveis de entrada e a variável de saída. Não estarei realizando uma extração de atributos, pois como cada modelo estará utilizando um algoritmo que parte de árvores de decisão, o próprio modelo estará selecionando as variáveis mais relevantes.

In [88]:
# Separando as variáveis explicativas e a variável alvo

X = df.drop(['ID', 'Segmentation'], axis=1)
y = df['Segmentation']

## Exercício 1: Aplicando os modelos num problema multiclasse

Como estratégia de divisão do modelo em dados de treino e teste para sua avaliação, estarei utilizando a validação cruzada com k=5 folds.

E como métrica, usarei o F1-score ponderado, pois leva em consideração dois scores que tratam a proporção de verdadeiros positivos de formas diferentes (a precisão e o recall) em uma média harmônica. Além disso, é considerado um score útil quando a quantidade de classes da previsão é diferente. Podemos verificar que há quantidades diferentes de fato.

In [105]:
y.value_counts()

D    2378
A    2304
C    2095
B    2016
Name: Segmentation, dtype: int64

Criando os modelos de classificação de árvore, baggind, AdaBoost e RandomForest:

In [99]:
# Criando o modelo de árvore de decisão
tree = DecisionTreeClassifier()
scores = cross_val_score(tree, X, y, scoring='f1_weighted').mean()
print(scores)

0.3739506804895071


In [98]:
# Criando o modelo bagging
bagging = BaggingClassifier(DecisionTreeClassifier())
scores = cross_val_score(bagging, X, y, cv=5, scoring='f1_weighted').mean()
print(scores)

0.40235473116220133


In [100]:
# Criando o modelo AdaBoost
adaboost = AdaBoostClassifier()
scores = cross_val_score(adaboost, X, y, cv=5, scoring='f1_weighted').mean()
print(scores)

0.45358375913633003


In [101]:
# Criando o modelo RandomForest
rforest = RandomForestClassifier()
scores = cross_val_score(rforest, X, y, cv=5, scoring='f1_weighted').mean()
print(scores)

0.41216263142323817


Dentre os modelos apresentados, o que teve melhor desempenho foi o AdaBoosting. Veremos como podemos melhorar seus hiperparâmetros:

In [103]:
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.5, 1.0]
}

# Realize a pesquisa em grade
grid_search = GridSearchCV(adaboost, param_grid, cv=5)
grid_search.fit(X, y)

# Exiba os melhores hiperparâmetros encontrados
best_params = grid_search.best_params_
print("Melhores hiperparâmetros:", best_params)

Melhores hiperparâmetros: {'learning_rate': 0.1, 'n_estimators': 150}


Em seguida, aplicamos os hiperparâmetros obtidos no modelo:

In [104]:
adaboost = AdaBoostClassifier(n_estimators=150,learning_rate=0.1)
scores = cross_val_score(adaboost, X, y, cv=5, scoring='f1_weighted').mean()
print(scores)

0.4569590551256429


## Exercício 2: Decompondo o problema em classificação binária

Vamos decompor o problema multiclasse em um problema de classificação binária por dois métodos diferentes, OVA (one-versus-all) e OVO (one-versus-one). Em ambos os casos, usaremos apenas o modelo de árvore de decisão e iremos realizar uma avaliação de cada um seguindo o mesmo procedimento de antes.

In [114]:
# Pelo OVA
OVA = OneVsRestClassifier(DecisionTreeClassifier())
scores = cross_val_score(OVA, X, y, cv=5).mean()
print(scores)


0.36664719909504895


In [115]:
# Pelo OVO
OVO = OneVsOneClassifier(DecisionTreeClassifier())
scores = cross_val_score(OVA, X, y, cv=5).mean()
print(scores)

0.3638048042862289


Podemos observar que neste caso de estudo, ambas as metodologias apresentam praticamente o mesmo desempenho.